## TERRE - VENT - FEU  - EAU - DATA ##





In [1]:
### Importation des librairies ###
import pandas as pd
import numpy as np 
import os
from pathlib import Path 
from datetime import datetime
from dotenv import load_dotenv
from sqlalchemy import create_engine, text
import psycopg2

In [2]:
### Connexion à la base de données ###
load_dotenv()
DB_URL = os.getenv('DATABASE_URL')
engine = create_engine(DB_URL)

### Requête SQL pour récupérer les données ###
## Données Incendies ##
# Liste des colonnes à récupérer
colonne_recup = ['annee', 'numero','code_insee', 'nom_commune', 'date_premiere_alerte', 'surface_parcourue_m2', 'type_de_peuplement','precision_de_la_donnee']

# Convertir la liste Python en string SQL
colonnes_sql = ', '.join(colonne_recup)

# Requête SQL
df = f"""
SELECT {colonnes_sql}
FROM incendies
"""
### Charger les données dans un DataFrame pandas ###
with engine.connect() as connection:
    df_incendies = pd.read_sql(text(df), connection)

## Données communes ##
# Liste des colonnes à récupérer
colonne_recup_communes = ['code_insee', 'nom_standard', 'population', 'superficie_km2', 'densite', 'latitude_centre', 'longitude_centre']

# Convertir la liste Python en string SQL
colonnes_sql_communes = ', '.join(colonne_recup_communes)

# Requête SQL
df_communes = f"""
SELECT {colonnes_sql_communes}
FROM communes
"""
### Charger les données dans un DataFrame pandas ###
with engine.connect() as connection:
    df_communes = pd.read_sql(text(df_communes), connection)

In [3]:
### Information sur les DataFrames ###
print(df_incendies.info())
print(df_communes.info())

## Pourcentage de valeurs manquantes dans df_incendies par colonnes ##
missing_percentage_incendies = df_incendies.isnull().mean() * 100
missing_values_incendies = df_incendies.isnull().sum()
print("Pourcentage et Nomnbres de valeurs manquantes dans df_incendies par colonnes :")
print(missing_percentage_incendies, missing_values_incendies)

## Pourcentage de valeurs manquantes dans df_communes par colonnes ##
missing_percentage_communes = df_communes.isnull().mean() * 100
missing_values_communes = df_communes.isnull().sum()
print("Pourcentage et Nombres de valeurs manquantes dans df_communes par colonnes :")
print(missing_percentage_communes, missing_values_communes)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140247 entries, 0 to 140246
Data columns (total 8 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   annee                   140247 non-null  int64         
 1   numero                  140247 non-null  object        
 2   code_insee              140247 non-null  object        
 3   nom_commune             140220 non-null  object        
 4   date_premiere_alerte    140247 non-null  datetime64[ns]
 5   surface_parcourue_m2    140247 non-null  int64         
 6   type_de_peuplement      77515 non-null   object        
 7   precision_de_la_donnee  1214 non-null    object        
dtypes: datetime64[ns](1), int64(2), object(5)
memory usage: 8.6+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35141 entries, 0 to 35140
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0  

In [4]:
### Supprimer ligne sans nom_standard dans df_communes ###
df_communes = df_communes.dropna(subset=['nom_standard'])

###  Changer les code_insee des noms de communes erronés dans df_incendies ###
# Dictionnaire de correspondance des codes erronés et des codes corrects
corrections_insee = {
    '05067': '05132',
    '05141': '05039',
    '05020': '50419',
    '48049': '48099',
    '48162': '48166',
    '05034': '05118',
    '48022': '48050',
    '48183': '48009',
    '48040': '48027',
    '48195': '48094',
    '48189': '48127',
    '48076': '48009',
    '48197': '48127',
    '48120': '48087',
    '48184': '48038',
    '48154': '48094',
    '27676': '27676',
    '07256': '07165'
}

# Appliquer les corrections dans df_incendies
df_incendies['code_insee'] = df_incendies['code_insee'].replace(corrections_insee)

# Vérifier que les noms de code_insee sont bien associés aux bons noms de communes
df_incendies = df_incendies.merge(df_communes[['code_insee', 'nom_standard']], on='code_insee', how='left', suffixes=('', '_corrected'))    
df_incendies['nom_commune'] = df_incendies['nom_standard'].combine_first(df_incendies['nom_commune'])
df_incendies = df_incendies.drop(columns=['nom_standard'])

# Si pas de correspondance trouvée, print le code_insee et le nom de la commune
mismatched = df_incendies[df_incendies['nom_commune'].isnull()][['code_insee', 'nom_commune']]
if not mismatched.empty:
    print("Codes INSEE sans correspondance trouvée :")
    print(mismatched)

## Print les lignes sans latitude_centre ##
missing_latitude = df_communes[df_communes['latitude_centre'].isnull()]

Commune_sans_lat = missing_latitude['nom_standard'].tolist()
coords = {
    "Marseille": (43.296398, 5.370000),
    "Culey": (48.75, 5.26667),
    "Les Hauts-Talican": (49.3294030, 2.0038440),
    "Lyon": (45.763420, 4.834277),
    "Paris": (48.864716, 2.349014),
    "Bihorel": (49.4547, 1.11611),
    "Saint-Lucien": (48.649, 1.626),
    "L'Oie": (46.7980, -1.1295),
    "Sainte-Florence": (46.796, 1.149 ),
}

## Ajouter les coordonnées manquantes dans df_communes ###
for commune, (lat, lon) in coords.items():
    df_communes.loc[df_communes['nom_standard'] == commune, 'latitude_centre'] = lat
    df_communes.loc[df_communes['nom_standard'] == commune, 'longitude_centre'] = lon

 ## Changer datetime to timestamp ##
df_incendies['date_premiere_alerte'] = pd.to_datetime(df_incendies['date_premiere_alerte'], errors='coerce').apply(
    lambda x: x.timestamp() if pd.notna(x) else None)

# Compter les lignes AVANT suppression
nb_lignes_initial = len(df_incendies)

# Créer colonne temporaire et supprimer doublons
df_incendies['date_jour'] = df_incendies['date_premiere_alerte'].apply(
    lambda x: datetime.fromtimestamp(x).date() if pd.notna(x) else None
)

df_incendies = df_incendies.drop_duplicates(
    subset=['date_jour', 'code_insee', 'surface_parcourue_m2'],
    keep='first'
).drop('date_jour', axis=1)

# Compter les lignes APRÈS suppression
nb_lignes_final = len(df_incendies)

# Calculer les statistiques
nb_supprimees = nb_lignes_initial - nb_lignes_final
pourcentage_supprime = (nb_supprimees / nb_lignes_initial) * 100
pourcentage_garde = (nb_lignes_final / nb_lignes_initial) * 100

# Affichage des résultats
print("=== NETTOYAGE DES DOUBLONS ===")
print(f"Lignes initiales    : {nb_lignes_initial:,}")
print(f"Lignes finales      : {nb_lignes_final:,}")
print(f"Lignes supprimées   : {nb_supprimees:,}")
print(f"Pourcentage supprimé : {pourcentage_supprime:.2f}%")

## Noter la localisation de chaque feu dans df_incendies grâce au coordonnées latitude et longitude du centre de chaque commune dans df_communes ##
df_incendies = df_incendies.merge(
    df_communes[['code_insee', 'latitude_centre', 'longitude_centre']],
    on='code_insee',
    how='left'
)
print(df_incendies)

=== NETTOYAGE DES DOUBLONS ===
Lignes initiales    : 140,247
Lignes finales      : 137,028
Lignes supprimées   : 3,219
Pourcentage supprimé : 2.30%
        annee numero code_insee       nom_commune  date_premiere_alerte  \
0        1973      8      06060           Falicon          9.543540e+07   
1        1973   1871      83038     Châteaudouble          9.707550e+07   
2        1973   1872      83078  Moissac-Bellevue          9.715410e+07   
3        1973   1639      34163        Montarnaud          9.716040e+07   
4        1973      9      06012        Beausoleil          9.716760e+07   
...       ...    ...        ...               ...                   ...   
137023   2024  29813      30040           Blandas          1.735389e+09   
137024   2024  25781      06074         Lantosque          1.735414e+09   
137025   2024  26495      97421           Salazie          1.735472e+09   
137026   2024  25459      34040            Brenas          1.735477e+09   
137027   2024  27169      0

In [5]:
print(df_incendies['latitude_centre'].isnull().sum())

## Print les lignes sans latitude_centre ##
missing_latitude_incendies = df_incendies[df_incendies['latitude_centre'].isnull()]
print("Lignes dans df_incendies sans latitude_centre :")
print(missing_latitude_incendies)

680
Lignes dans df_incendies sans latitude_centre :
        annee numero code_insee                  nom_commune  \
909      1973    382      11298             Pradelles-en-Val   
996      1973    236      07252       Saint-Julien-Boutières   
1902     1973    459      11050                       Brenac   
2093     1973    508      11097      Conilhac-de-la-Montagne   
2097     1973   1601      30157                         Mars   
...       ...    ...        ...                          ...   
123523   2020  18749      48172    Saint-Maurice-de-Ventalon   
123785   2020  12602      48134  Saint-Andéol-de-Clerguemort   
124474   2020  15538      48078                      Lachamp   
125775   2020  16294      11329                     Rouvenac   
131440   2022   2504      71492                Saint-Ythaire   

        date_premiere_alerte  surface_parcourue_m2 type_de_peuplement  \
909             1.122768e+08                 25000               None   
996             1.127178e+08     